In [1]:
# very confusing code
# im not going to document it
# check 8AAFFF on yt for explanation

In [2]:
import torch
import pygame

pygame 2.5.1 (SDL 2.28.2, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
# world initialization settings:
simulation_dimension = 2
num_of_particles = 400
# particles will be instantiated from -1000 to 1000
scatter_size = 1000

# particle interaction settings:
big_G = 0.1
particle_mass = 0.01
momentum_conservation_value = 0.99

# distance <---> gravity function modifiers
gravity_r = 100
gravity_c = 0.01
gravity_f = 0
gravity_u = 0.001

# same thing here, ior function modifiers.
# here i tried making nearby particles apply similar forces on each other to simulate interstellar gas.

# no longer functional im afraid to delete

# distance <---> gas function modifiers
#smoothing_rate = 0.8

# misc settings:
# if u want to add a bunch of particles strapped to ur mouse.
# 0 -> disable thing
center_black_hole_number = 0#1000

In [4]:
# if no gpu
GPU = torch.device("cpu")#torch.device("cuda")
world = (2 * scatter_size * torch.rand((num_of_particles + center_black_hole_number) * simulation_dimension, dtype=torch.float32, device=GPU) - scatter_size).view(-1, simulation_dimension)
#world[:center_black_hole_number] = torch.zeros(world[:center_black_hole_number].shape)

In [5]:
# func see desmos
# \frac{r\left(ux-f\right)}{\left(ux-f\right)^{2}+c}
# ^^ copy paste to desmos
# works on tensors and python scalars
def distance_decoder_function(intake, r, c, f, u):
    return (r * (u*intake-f)) / ((u*intake-f) ** 2 + c)

In [6]:
# for feature to give close partices similar forces
def smooth_tensor(tnsr):
    score = torch.sum(world, dim=1)
    
    sorted_indices = torch.argsort(score)
    original_indices = torch.argsort(sorted_indices)
    
    sorted_tnsr = tnsr[sorted_indices]
    blurred_tnsr = (sorted_tnsr + torch.cat((sorted_tnsr[-1:], sorted_tnsr[:-1]), dim=0)) / 2
    unsorted_blurred_tnsr = blurred_tnsr[original_indices]
    
    return unsorted_blurred_tnsr

In [7]:
# actual calculation to porgress simulation by one frame
def get_global_vectors(world, previous_global_vectors):
    differences = world[:, None, :] - world[None, :, :]
    squared_distances = torch.sum(differences**2, dim=-1)
    inverse_squared_distances = distance_decoder_function(squared_distances, gravity_r, gravity_c, gravity_f, gravity_u)
    
    inverse_squared_distances[torch.isinf(inverse_squared_distances)] = 0.0

    gravity_forces = -big_G * differences * particle_mass * inverse_squared_distances[:, :, None]
    gravity_momentum_forces = gravity_forces.sum(dim=1) + previous_global_vectors * momentum_conservation_value
    
    #gravity_momentum_forces = smooth_tensor(gravity_momentum_forces)
    
    return gravity_momentum_forces

In [8]:
# i aint documenting all that
# ask chatgpt bcs it wrote it anyways




# Initialize Pygame
pygame.init()

# Set the window dimensions
window_size = (800, 800)

# Create a window
screen = pygame.display.set_mode(window_size)
pygame.display.set_caption("Particle Simulation")

# Define colors
bg_color = (0, 0, 0)
color = (255, 255, 255)

# Function to map PyTorch tensor positions to screen coordinates
def map_to_screen(position, max_position): 
    x, y = position.detach().cpu().numpy()
    x = int((x / max_position + 1) * window_size[0] / 2)
    y = int((y / max_position + 1) * window_size[1] / 2)
    return x, y

def map_from_screen(position, max_position):
    x, y = position[0], position[1]
    x = (x / window_size[0] * 2 - 1) * max_position
    y = (y / window_size[1] * 2 - 1) * max_position
    return torch.tensor((x, y), dtype=torch.float32, device=GPU)

# Main game loop
running = True
vectors = torch.zeros(world.shape, dtype=torch.float32, device=GPU)# * 700 - 350
max_position = 0

while running:
    if float(torch.max(torch.abs(world))) >= max_position:
        max_position = float(torch.max(torch.abs(world)))
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    # Clear the screen
    screen.fill(bg_color)
    
    #if torch.relu(scatter_size - torch.max(torch.abs(world))) > 0:
    #    world += (world * torch.abs(world) / world ** 2) * torch.relu(scatter_size - torch.max(torch.abs(world)))
    
    # Update your PyTorch world (you'll need to adapt this part)
    # ---+---
    vectors = get_global_vectors(world, vectors)
    world += vectors
    world[:center_black_hole_number] = torch.ones(world[:center_black_hole_number].shape) * map_from_screen(pygame.mouse.get_pos(), max_position)
    
    if world.shape[1] < 2:
        zeros = torch.zeros((world.shape[0], 2 - world.shape[1]), device=GPU)
        clipped = torch.cat((world, zeros), dim=1)
    else:
        clipped = world[:, :2]
    #print(pygame.mouse.get_pos())
    counter = 0
    # Draw dots on the screen based on your PyTorch world
    for position in world:
        # dims 0->2
        if simulation_dimension >= 2:
            position = map_to_screen(world[counter][:2], max_position)
        else:
            zeros = torch.zeros((world.shape[0], 2 - world.shape[1]), device=GPU)
            position = map_to_screen(torch.cat((world, zeros), dim=1), max_position)
        
        # special dim 3:
        if simulation_dimension == 3:
            min_val = torch.min(world[:, 2])
            max_val = torch.max(world[:, 2])
            color = torch.clamp(((world[counter][2] - min_val) / (max_val - min_val) * 255).repeat(3), 50, 300).tolist()

            #print(shade)
            #color = shade
        
        # dims 2->5
        if simulation_dimension >= 5:
            color = ((world[counter][2:5] - world[counter][2:5].min()) / (world[counter][2:5].max() - world[counter][2:5].min()) * 255).long().tolist()
        
        pygame.draw.circle(screen, color, position, 1)#float(torch.sigmoid(torch.mean(world[counter][2:])) * particle_display_size + 1))  # 5 is the radius of the dot
        counter += 1
    pygame.draw.circle(screen, (116, 255, 56), map_to_screen(map_from_screen(pygame.mouse.get_pos(), max_position), max_position), 5)  # 5 is the radius of the dot

    # Update the display
    pygame.display.flip()

# Quit Pygame
pygame.quit()
sys.exit()

NameError: name 'sys' is not defined